In [ ]:
%pip install datasets transformers --quiet

In [ ]:
import pandas as pd
import numpy as np
import random
import json

seed = 42

np.random.seed(seed)
random.seed(seed)

Getting the dataset.

In [ ]:
from datasets import load_dataset
sem_eval_2018_task_1 = load_dataset('sem_eval_2018_task_1', 'subtask5.english', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

For each of the files with generated GPT data, we parse the JSON file, drop the missing values and, if necessary, convert the label columns to boolean values. The data is combined with half of the gold standard data. Additionally, the distribution of labels can be verified.

In [ ]:
generated_df = pd.read_csv('generated_tweets_no_role.csv')

In [ ]:
# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data.drop(columns=["Tweet"]).sum()

,0
anger,104
anticipation,723
disgust,158
fear,256
joy,600
love,399
optimism,610
pessimism,495
sadness,296
surprise,387


Half GSD

In [ ]:
# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data.csv', index=False)

Full GSD data

In [ ]:
# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_no_role_news.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_no_role_news.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_no_role_news.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Concatenate df and augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data_no_role_news.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_1.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_1.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_1.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data_role_1.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_2.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_2.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_2.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data_role_2.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_3.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_3.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_3.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data_role_3.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_4.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_4.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_4.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data_role_4.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_5.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_5.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_5.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_multi_gpt_data_role_5.csv', index=False)

## Single-label Data

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_no_role.csv')

# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_single_label_no_role.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_no_role.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_single_gpt_data_no_role.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_role_1.csv')

# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_single_label_role_1.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_role_1.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_single_gpt_data_role_1.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_role_1_news.csv')

# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_single_label_role_1_news.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_role_1_news.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_single_gpt_data_role_1_news.csv', index=False)

## Increased size data

In [ ]:
# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_increased_multi.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_increased_multi.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_increased_multi.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_increased_gpt_data_multi.csv', index=False)

Half GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_increased_single.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_increased_single.csv', index=False)

Full GSD

In [ ]:
generated_df = pd.read_csv('generated_tweets_increased_single.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('full_gsd_increased_gpt_data_single.csv', index=False)

## Full GSD + GPT Data
Generating data for multi-label data, single-label data and increased size data.

In [ ]:
# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

## DATA Typicality

Inter-dataset typicality.

We are looking at how similar the tweets from the generated data are to the original gold standard data tweets.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_no_role.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.96 with standard deviation of 0.10
Typicality for anticipation: 0.96 with standard deviation of 0.10
Typicality for disgust: 0.96 with standard deviation of 0.11
Typicality for fear: 0.98 with standard deviation of 0.12
Typicality for joy: 1.03 with standard deviation of 0.14
Typicality for love: 1.08 with standard deviation of 0.23
Typicality for optimism: 1.13 with standard deviation of 0.16
Typicality for pessimism: 1.13 with standard deviation of 0.13
Typicality for sadness: 1.12 with standard deviation of 0.15
Typicality for surprise: 0.92 with standard deviation of 0.13
Typicality for trust: 1.08 with standard deviation of 0.13

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_no_role_news.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 1.00 with standard deviation of 0.08
Typicality for anticipation: 1.01 with standard deviation of 0.09
Typicality for disgust: 1.01 with standard deviation of 0.08
Typicality for fear: 1.03 with standard deviation of 0.08
Typicality for joy: 0.97 with standard deviation of 0.09
Typicality for love: 0.96 with standard deviation of 0.19
Typicality for optimism: 1.09 with standard deviation of 0.12
Typicality for pessimism: 1.10 with standard deviation of 0.10
Typicality for sadness: 1.07 with standard deviation of 0.11
Typicality for surprise: 0.98 with standard deviation of 0.20
Typicality for trust: 1.06 with standard deviation of 0.12

Overall Typicality: 1.02


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_1.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.98 with standard deviation of 0.12
Typicality for anticipation: 0.98 with standard deviation of 0.11
Typicality for disgust: 0.98 with standard deviation of 0.11
Typicality for fear: 1.01 with standard deviation of 0.13
Typicality for joy: 0.99 with standard deviation of 0.12
Typicality for love: 1.01 with standard deviation of 0.21
Typicality for optimism: 1.12 with standard deviation of 0.15
Typicality for pessimism: 1.14 with standard deviation of 0.13
Typicality for sadness: 1.11 with standard deviation of 0.14
Typicality for surprise: 0.91 with standard deviation of 0.12
Typicality for trust: 1.07 with standard deviation of 0.15

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_2.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.95 with standard deviation of 0.12
Typicality for anticipation: 0.98 with standard deviation of 0.12
Typicality for disgust: 0.95 with standard deviation of 0.12
Typicality for fear: 0.98 with standard deviation of 0.12
Typicality for joy: 1.03 with standard deviation of 0.16
Typicality for love: 1.08 with standard deviation of 0.27
Typicality for optimism: 1.16 with standard deviation of 0.18
Typicality for pessimism: 1.13 with standard deviation of 0.13
Typicality for sadness: 1.10 with standard deviation of 0.14
Typicality for surprise: 0.91 with standard deviation of 0.14
Typicality for trust: 1.09 with standard deviation of 0.15

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_3.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.95 with standard deviation of 0.11
Typicality for anticipation: 0.98 with standard deviation of 0.11
Typicality for disgust: 0.95 with standard deviation of 0.12
Typicality for fear: 0.99 with standard deviation of 0.13
Typicality for joy: 1.03 with standard deviation of 0.15
Typicality for love: 1.08 with standard deviation of 0.25
Typicality for optimism: 1.16 with standard deviation of 0.17
Typicality for pessimism: 1.14 with standard deviation of 0.15
Typicality for sadness: 1.11 with standard deviation of 0.17
Typicality for surprise: 0.91 with standard deviation of 0.14
Typicality for trust: 1.09 with standard deviation of 0.15

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_4.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.98 with standard deviation of 0.11
Typicality for anticipation: 0.97 with standard deviation of 0.11
Typicality for disgust: 0.99 with standard deviation of 0.11
Typicality for fear: 1.01 with standard deviation of 0.12
Typicality for joy: 0.96 with standard deviation of 0.10
Typicality for love: 0.97 with standard deviation of 0.18
Typicality for optimism: 1.10 with standard deviation of 0.13
Typicality for pessimism: 1.16 with standard deviation of 0.13
Typicality for sadness: 1.14 with standard deviation of 0.14
Typicality for surprise: 0.91 with standard deviation of 0.16
Typicality for trust: 1.02 with standard deviation of 0.13

Overall Typicality: 1.02


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_5.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 1.00 with standard deviation of 0.09
Typicality for anticipation: 1.01 with standard deviation of 0.09
Typicality for disgust: 1.01 with standard deviation of 0.09
Typicality for fear: 1.03 with standard deviation of 0.09
Typicality for joy: 0.96 with standard deviation of 0.09
Typicality for love: 0.95 with standard deviation of 0.18
Typicality for optimism: 1.08 with standard deviation of 0.13
Typicality for pessimism: 1.11 with standard deviation of 0.11
Typicality for sadness: 1.07 with standard deviation of 0.12
Typicality for surprise: 0.98 with standard deviation of 0.21
Typicality for trust: 1.05 with standard deviation of 0.12

Overall Typicality: 1.02


Single-label

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_single_label_no_role.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.95 with standard deviation of 0.11
Typicality for anticipation: 0.97 with standard deviation of 0.11
Typicality for disgust: 0.96 with standard deviation of 0.10
Typicality for fear: 0.98 with standard deviation of 0.11
Typicality for joy: 1.03 with standard deviation of 0.14
Typicality for love: 1.07 with standard deviation of 0.23
Typicality for optimism: 1.13 with standard deviation of 0.17
Typicality for pessimism: 1.14 with standard deviation of 0.17
Typicality for sadness: 1.13 with standard deviation of 0.19
Typicality for surprise: 0.91 with standard deviation of 0.12
Typicality for trust: 1.08 with standard deviation of 0.13

Overall Typicality: 1.03


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_single_label_role_1.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.99 with standard deviation of 0.12
Typicality for anticipation: 0.98 with standard deviation of 0.12
Typicality for disgust: 0.99 with standard deviation of 0.12
Typicality for fear: 0.99 with standard deviation of 0.12
Typicality for joy: 0.99 with standard deviation of 0.12
Typicality for love: 1.01 with standard deviation of 0.21
Typicality for optimism: 1.11 with standard deviation of 0.15
Typicality for pessimism: 1.12 with standard deviation of 0.12
Typicality for sadness: 1.10 with standard deviation of 0.14
Typicality for surprise: 0.92 with standard deviation of 0.12
Typicality for trust: 1.07 with standard deviation of 0.14

Overall Typicality: 1.03


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_single_label_role_1_news.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 1.00 with standard deviation of 0.09
Typicality for anticipation: 1.01 with standard deviation of 0.10
Typicality for disgust: 1.01 with standard deviation of 0.10
Typicality for fear: 1.02 with standard deviation of 0.09
Typicality for joy: 0.97 with standard deviation of 0.10
Typicality for love: 0.95 with standard deviation of 0.20
Typicality for optimism: 1.08 with standard deviation of 0.13
Typicality for pessimism: 1.10 with standard deviation of 0.13
Typicality for sadness: 1.07 with standard deviation of 0.16
Typicality for surprise: 0.98 with standard deviation of 0.20
Typicality for trust: 1.05 with standard deviation of 0.14

Overall Typicality: 1.02


Additionally, we compare the typocality of labels in the first generated dataset, as well as comparing generated datasets to each other.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json

# Specify the dataset to compare
dataset_name = 'role_1'
filepath = 'generated_tweets_role_1.csv'

# Load the dataset
generated_data = pd.read_csv(filepath)

# Convert JSON strings to dictionaries if needed
if isinstance(generated_data.iloc[0, 0], str) and generated_data.iloc[0, 0].startswith('{'):
    parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]
    generated_data = pd.DataFrame(parsed_data)
    generated_data = generated_data.dropna()

# Extract tweets and emotion labels
tweets = generated_data['Tweet'].tolist()
emotion_labels = generated_data.drop(columns=['Tweet'])

# Fit vectorizer on this dataset's text only
vectorizer = TfidfVectorizer()
vec = vectorizer.fit_transform(tweets)

# Calculate cosine similarity of the dataset with itself
similarity_matrix = cosine_similarity(vec, vec)

# Function to calculate typicality for each emotion
def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = emotion_labels.to_numpy()

# Dictionary to store typicality scores for each emotion
emotion_typicality_scores = {}

# Calculate typicality for each emotion
for i, emotion in enumerate(emotion_labels.columns):
    typicality_scores = calculate_typicality(similarity_matrix, labels, i)
    emotion_typicality_scores[emotion] = np.mean(typicality_scores)

# Print typicality scores for each emotion
for emotion, score in emotion_typicality_scores.items():
    print(f"Typicality for {emotion} in {dataset_name}: {score:.2f}")

# Print overall typicality score
overall_typicality = np.mean(list(emotion_typicality_scores.values()))
print(f"Overall Typicality for {dataset_name}: {overall_typicality:.2f}")

Typicality for anger in role_1: 1.12
Typicality for anticipation in role_1: 1.56
Typicality for disgust in role_1: 1.01
Typicality for fear in role_1: 1.36
Typicality for joy in role_1: 1.59
Typicality for love in role_1: 1.22
Typicality for optimism in role_1: 1.61
Typicality for pessimism in role_1: 1.66
Typicality for sadness in role_1: 1.69
Typicality for surprise in role_1: 1.24
Typicality for trust in role_1: 1.44
Overall Typicality for role_1: 1.41


Ad

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json

# List of generated datasets
generated_datasets = {
    'no_role': 'generated_tweets_no_role.csv',
    'no_role_news': 'generated_tweets_no_role_news.csv',
    'role_1': 'generated_tweets_role_1.csv',
    'role_2': 'generated_tweets_role_2.csv',
    'role_3': 'generated_tweets_role_3.csv',
    'role_4': 'generated_tweets_role_4.csv',
    'role_5': 'generated_tweets_role_5.csv',
}

# Load all generated datasets and combine text for vectorizer fitting
all_texts = []
dataset_texts = {}
dataset_labels = {}

for name, filepath in generated_datasets.items():
    generated_data = pd.read_csv(filepath)

    # Convert JSON strings to dictionaries if needed
    if isinstance(generated_data.iloc[0, 0], str) and generated_data.iloc[0, 0].startswith('{'):
        parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]
        generated_data = pd.DataFrame(parsed_data)
        generated_data = generated_data.dropna()

    tweets = generated_data['Tweet'].tolist()
    labels = generated_data.drop(columns=['Tweet'])  # Emotion labels

    all_texts.extend(tweets)  # Collect all texts for fitting vectorizer
    dataset_texts[name] = tweets  # Store each dataset's texts separately
    dataset_labels[name] = labels.to_numpy()  # Store emotion labels separately

# Fit vectorizer on combined text
vectorizer = TfidfVectorizer()
vectorizer.fit(all_texts)

# Vectorize each dataset using the common vectorizer
vectorized_data = {name: vectorizer.transform(texts) for name, texts in dataset_texts.items()}

# Dictionary to store typicality results between datasets
typicality_results = {}

# Function to calculate typicality for each emotion
def calculate_typicality(similarities, labels, target_label):
    if target_label >= labels.shape[1]:  # Ensure target_label is within bounds
        return np.array([])

    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    if len(target_indices) == 0 or len(non_target_indices) == 0:
        return np.array([])  # Return empty array if no indices for target or non-target

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Compare each pair of datasets
for name1, vec1 in vectorized_data.items():
    for name2, vec2 in vectorized_data.items():
        if name1 != name2:
            # Align sample sizes
            min_samples = min(vec1.shape[0], vec2.shape[0])
            vec1_aligned = vec1[:min_samples]
            vec2_aligned = vec2[:min_samples]

            # Calculate cosine similarity between the two datasets
            similarity_matrix = cosine_similarity(vec1_aligned, vec2_aligned)

            # Align labels to the smallest number of samples
            labels1 = dataset_labels[name1][:min_samples]
            labels2 = dataset_labels[name2][:min_samples]

            # Calculate typicality scores for each emotion
            num_emotions = min(labels1.shape[1], labels2.shape[1])
            emotion_typicality_scores = {}
            for i in range(num_emotions):  # Iterate over common number of emotions
                typicality_scores = calculate_typicality(similarity_matrix, labels1, i)
                emotion_typicality_scores[f'{name1}_vs_{name2}_{i}'] = np.mean(typicality_scores) if len(typicality_scores) > 0 else np.nan

            # Store the average typicality score for all emotions
            valid_scores = [score for score in emotion_typicality_scores.values() if not np.isnan(score)]
            overall_typicality = np.mean(valid_scores) if len(valid_scores) > 0 else np.nan
            typicality_results[(name1, name2)] = overall_typicality

# Print overall typicality scores between each pair of generated datasets
for (name1, name2), score in typicality_results.items():
    print(f"Typicality between {name1} and {name2}: {score:.2f}")


Typicality between no_role and no_role_news: 1.05
Typicality between no_role and role_1: 1.26
Typicality between no_role and role_2: 1.35
Typicality between no_role and role_3: 1.36
Typicality between no_role and role_4: 1.13
Typicality between no_role and role_5: 1.05
Typicality between no_role_news and no_role: 1.08
Typicality between no_role_news and role_1: 1.06
Typicality between no_role_news and role_2: 1.07
Typicality between no_role_news and role_3: 1.08
Typicality between no_role_news and role_4: 1.05
Typicality between no_role_news and role_5: 1.05
Typicality between role_1 and no_role: 1.23
Typicality between role_1 and no_role_news: 1.04
Typicality between role_1 and role_2: 1.35
Typicality between role_1 and role_3: 1.35
Typicality between role_1 and role_4: 1.15
Typicality between role_1 and role_5: 1.05
Typicality between role_2 and no_role: 1.25
Typicality between role_2 and no_role_news: 1.04
Typicality between role_2 and role_1: 1.29
Typicality between role_2 and role

Gold standard data typicality

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Load training data

# Extract tweets and emotion labels
tweets = df['Tweet'].tolist()
emotion_labels = df.drop(columns=['Tweet', 'ID'])  # Emotion labels only

# Vectorize the tweets
vectorizer = TfidfVectorizer()
vec = vectorizer.fit_transform(tweets)

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(vec, vec)

# Function to calculate typicality for each emotion label
def calculate_typicality(similarity_matrix, labels, target_label_index):
    # Indices of tweets with and without the target emotion
    target_indices = np.where(labels[:, target_label_index] == 1)[0]
    non_target_indices = np.where(labels[:, target_label_index] == 0)[0]

    if len(target_indices) == 0 or len(non_target_indices) == 0:
        # If there are no tweets with or without the emotion, return NaN
        return np.nan

    # Initialize lists to store average similarities
    avg_sim_target = []
    avg_sim_non_target = []

    # Calculate average similarity for tweets with the target label
    for idx in target_indices:
        sim_with_target = similarity_matrix[idx, target_indices]  # Similarities with other target tweets
        avg_sim_target.append(np.mean(sim_with_target))

    # Calculate average similarity for tweets without the target label
    for idx in target_indices:
        sim_with_non_target = similarity_matrix[idx, non_target_indices]  # Similarities with non-target tweets
        avg_sim_non_target.append(np.mean(sim_with_non_target))

    # Calculate typicality score as the ratio of average similarities
    avg_sim_target_all = np.mean(avg_sim_target)
    avg_sim_non_target_all = np.mean(avg_sim_non_target)

    # Avoid division by zero
    avg_sim_non_target_all += 1e-10  # Small constant to prevent division by zero

    typicality = avg_sim_target_all / avg_sim_non_target_all
    return typicality

# Convert labels to a NumPy array for processing
labels = emotion_labels.to_numpy()

# Dictionary to store typicality scores for each emotion
emotion_typicality_scores = {}

# Calculate typicality for each emotion label
for i, emotion in enumerate(emotion_labels.columns):
    typicality_score = calculate_typicality(similarity_matrix, labels, i)
    emotion_typicality_scores[emotion] = typicality_score

# Print typicality scores for each emotion
for emotion, score in emotion_typicality_scores.items():
    print(f"Typicality for {emotion} in original dataset: {score:.2f}")

# Print overall typicality score
overall_typicality = np.mean(list(emotion_typicality_scores.values()))
print(f"Overall Typicality for original dataset: {overall_typicality:.2f}")

Typicality for anger in original dataset: 1.13
Typicality for anticipation in original dataset: 1.13
Typicality for disgust in original dataset: 1.11
Typicality for fear in original dataset: 1.25
Typicality for joy in original dataset: 1.13
Typicality for love in original dataset: 1.45
Typicality for optimism in original dataset: 1.22
Typicality for pessimism in original dataset: 1.25
Typicality for sadness in original dataset: 1.17
Typicality for surprise in original dataset: 1.37
Typicality for trust in original dataset: 1.37
Overall Typicality for original dataset: 1.23


## Selecting examples of generated tweets

In [ ]:
def process_generated_data(generated_df):
    """
    Processes the DataFrame containing JSON strings in the first column.

    Parameters:
    generated_df (pd.DataFrame): DataFrame with JSON strings in the first column.

    Returns:
    pd.DataFrame: Processed DataFrame with boolean values for label columns.
    """
    # Convert the JSON strings in the first column to dictionaries
    parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

    # Create a new DataFrame from the parsed data
    generated_data = pd.DataFrame(parsed_data)

    # Drop rows with missing values
    generated_data = generated_data.dropna()
    generated_data = generated_data.drop_duplicates()

    # Define the list of label columns
    label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love',
                     'anger', 'disgust', 'pessimism', 'sadness', 'fear',
                     'surprise']

    # Convert label columns to boolean values
    generated_data[label_columns] = generated_data[label_columns].astype(bool)

    return generated_data

In [ ]:
# Example of loading data and using the function
generated_df = pd.read_csv('generated_tweets_role_5.csv')
processed_df = process_generated_data(generated_df)

Getting the multi-label examples

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_best_emotion_tweet(dataset, emotions):
    """
    This function takes a multi-label dataset and returns the best tweet for each specified emotion
    using cosine similarity, along with the average similarity score and associated labels.

    Parameters:
    dataset (pd.DataFrame): The loaded DataFrame.
    emotions (list of str): List of emotions to find the best tweet for.

    Returns:
    dict: A dictionary where keys are emotions and values are tuples containing
          the best tweet, its average similarity score, and its labels.
    """
    results = {}

    for emotion in emotions:
        # Initialize list to store tweets for the specified emotion
        emotion_tweets = []
        labels_list = []

        # Iterate through the dataset
        for _, row in dataset.iterrows():
            tweet = row['Tweet']

            # Add tweet to the list if it is labeled with the specified emotion
            if row[emotion] == 1:
                emotion_tweets.append(tweet)
                labels = {label: row[label] for label in emotions if row[label] == 1}
                labels_list.append(labels)

        if not emotion_tweets:
            results[emotion] = (None, None, None)
            continue

        # Vectorize the tweets using TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english')
        tweet_vectors = vectorizer.fit_transform(emotion_tweets)

        # Calculate cosine similarity matrix
        cosine_sim = cosine_similarity(tweet_vectors)

        # Average similarity score for each tweet
        avg_sim_scores = cosine_sim.mean(axis=1)

        # Get the index of the tweet with the highest average similarity
        best_idx = np.argmax(avg_sim_scores)
        best_tweet = emotion_tweets[best_idx]
        best_score = avg_sim_scores[best_idx]
        best_labels = labels_list[best_idx]

        results[emotion] = (best_tweet, best_score, best_labels)

    return results

# Define the list of emotions
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Get the best example for each specified emotion
best_tweets = get_best_emotion_tweet(processed_df, emotions)

# Output the best example with its cosine similarity score and associated labels for each emotion
for emotion, (best_tweet, best_score, best_labels) in best_tweets.items():
    if best_tweet:
        labels_str = ', '.join([f"{label}: {int(value)}" for label, value in best_labels.items()])
        print(f"Best example for {emotion}: {best_tweet} (Average Cosine Similarity: {best_score:.4f}) with labels: {labels_str}")
    else:
        print(f"No tweets found for emotion: {emotion}")

Best example for anger: Wow, Russia's media really called it 'the Olympics of Hell' after the Paris ban! Just shocking! 😲😡 (Average Cosine Similarity: 0.0570) with labels: anger: 1, surprise: 1
Best example for anticipation: Wow, a dead bear in RFK Jr's campaign? This is just wild! Can't believe the twists we see in politics. 😳 (Average Cosine Similarity: 0.0445) with labels: anticipation: 1, pessimism: 1, surprise: 1
Best example for disgust: Hearing about the US funeral home fined $950m for decaying bodies is terrifying. How can people trust our systems? (Average Cosine Similarity: 0.0683) with labels: anger: 1, disgust: 1, fear: 1, pessimism: 1, sadness: 1, surprise: 1
Best example for fear: Just heard about that small plane crash on the golf course! How terrifying! 😱 (Average Cosine Similarity: 0.0560) with labels: anticipation: 1, fear: 1, pessimism: 1, surprise: 1
Best example for joy: Exciting times in football! Atletico's £81.5m deal for Alvarez shows ambition and hope. Can't w

For single-label

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_best_emotion_tweet(dataset, emotions):
    """
    This function takes a single-label dataset and returns the best tweet for each specified emotion
    using cosine similarity, along with the average similarity score.

    Parameters:
    dataset (pd.DataFrame): The loaded DataFrame.
    emotions (list of str): List of emotions to find the best tweet for.

    Returns:
    dict: A dictionary where keys are emotions and values are tuples containing
          the best tweet and its average similarity score.
    """
    results = {}

    for emotion in emotions:
        # Initialize list to store tweets for the specified emotion
        emotion_tweets = []

        # Iterate through the dataset
        for _, row in dataset.iterrows():
            tweet = row['Tweet']

            # Add tweet to the list if it is labeled with the specified emotion
            if row[emotion] == 1:
                emotion_tweets.append(tweet)

        if not emotion_tweets:
            results[emotion] = (None, None)
            continue

        # Vectorize the tweets using TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english')
        tweet_vectors = vectorizer.fit_transform(emotion_tweets)

        # Calculate cosine similarity matrix
        cosine_sim = cosine_similarity(tweet_vectors)

        # Average similarity score for each tweet
        avg_sim_scores = cosine_sim.mean(axis=1)

        # Get the index of the tweet with the highest average similarity
        best_idx = np.argmax(avg_sim_scores)
        best_tweet = emotion_tweets[best_idx]
        best_score = avg_sim_scores[best_idx]

        results[emotion] = (best_tweet, best_score)

    return results

# Define the list of emotions
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Load and process the data
generated_df = pd.read_csv('generated_tweets_single_label_role_1_news.csv')
processed_df = process_generated_data(generated_df)

# Get the best example for each specified emotion
best_tweets = get_best_emotion_tweet(processed_df, emotions)

# Output the best example with its cosine similarity score for each emotion
for emotion, (best_tweet, best_score) in best_tweets.items():
    if best_tweet:
        print(f"Best example for {emotion}: {best_tweet} (Average Cosine Similarity: {best_score:.4f})")
    else:
        print(f"No tweets found for emotion: {emotion}")

Best example for anger: Can you believe the parliament dissolved just like that? This is beyond frustrating! (Average Cosine Similarity: 0.0784)
Best example for anticipation: Flintoff's 'nightmares' from the Top Gear crash sound intense. I can't wait to hear more about this! (Average Cosine Similarity: 0.0599)
Best example for disgust: Protests show some anger, but honestly, this situation is just frustrating. It feels like a lot of noise for nothing. (Average Cosine Similarity: 0.0556)
Best example for fear: This news about the funeral home is terrifying. How could this happen? (Average Cosine Similarity: 0.0744)
Best example for joy: Exciting times ahead as President Biden gathers his national security team! Let's hope for a peaceful resolution. #Hope #Peace (Average Cosine Similarity: 0.0552)
Best example for love: @economicinsights I know times are tough, but let’s remember to support each other. Love and unity can guide us through! (Average Cosine Similarity: 0.0705)
Best example

## Type-Token Ratio

In [14]:
def process_generated_data(generated_df):
    """
    Processes the DataFrame containing JSON strings in the first column.

    Parameters:
    generated_df (pd.DataFrame): DataFrame with JSON strings in the first column.

    Returns:
    pd.DataFrame: Processed DataFrame with boolean values for label columns.
    """
    # Convert the JSON strings in the first column to dictionaries
    parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

    # Create a new DataFrame from the parsed data
    generated_data = pd.DataFrame(parsed_data)

    # Drop rows with missing values
    generated_data = generated_data.dropna()
    generated_data = generated_data.drop_duplicates()

    # Define the list of label columns
    label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love',
                     'anger', 'disgust', 'pessimism', 'sadness', 'fear',
                     'surprise']

    # Convert label columns to boolean values
    generated_data[label_columns] = generated_data[label_columns].astype(bool)

    return generated_data

In [17]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Download the tokenizer models
nltk.download('punkt')

# Define function to calculate TTR
def calculate_overall_ttr(tweets):
    all_tokens = []
    unique_tokens = set()

    # Tokenize each tweet and collect tokens
    for tweet in tweets:
        tokens = word_tokenize(tweet.lower())
        all_tokens.extend(tokens)
        unique_tokens.update(tokens)

    # Calculate total tokens and unique types
    total_tokens = len(all_tokens)
    unique_types = len(unique_tokens)

    # Calculate and return the overall TTR
    return unique_types / total_tokens if total_tokens > 0 else 0

# Average length of a tweet
def calculate_average_length_nltk(tweets):
    lengths = []

    for tweet in tweets:
      tokens = word_tokenize(tweet)  # Tokenize tweet
      length = len(tokens)  # Word count

      lengths.append(length)

    # Calculate average length
    average_length = sum(lengths) / len(lengths)
    return average_length

# datasets
df = pd.DataFrame(sem_eval_2018_task_1['train'])

df_no_role = pd.read_csv('generated_tweets_no_role.csv')
df_no_role = process_generated_data(df_no_role)

df_no_role_news = pd.read_csv('generated_tweets_no_role_news.csv')
df_no_role_news = process_generated_data(df_no_role_news)

df_role_1 = pd.read_csv('generated_tweets_role_1.csv')
df_role_1 = process_generated_data(df_role_1)

df_role_2 = pd.read_csv('generated_tweets_role_2.csv')
df_role_2 = process_generated_data(df_role_2)

df_role_3 = pd.read_csv('generated_tweets_role_3.csv')
df_role_3 = process_generated_data(df_role_3)

df_role_4 = pd.read_csv('generated_tweets_role_4.csv')
df_role_4 = process_generated_data(df_role_4)

df_role_5 = pd.read_csv('generated_tweets_role_5.csv')
df_role_5 = process_generated_data(df_role_5)

df_single_no_role = pd.read_csv('generated_tweets_single_label_no_role.csv')
df_single_no_role = process_generated_data(df_single_no_role)

df_single_role_1 = pd.read_csv('generated_tweets_single_label_role_1.csv')
df_single_role_1 = process_generated_data(df_single_role_1)

df_single_role_1_news = pd.read_csv('generated_tweets_single_label_role_1_news.csv')
df_single_role_1_news = process_generated_data(df_single_role_1_news)

# Apply the function for TTR
df_ttr = calculate_overall_ttr(df['Tweet'])
df_no_role_ttr = calculate_overall_ttr(df_no_role['Tweet'])
df_no_role_news_ttr = calculate_overall_ttr(df_no_role_news['Tweet'])
df_role_1_ttr = calculate_overall_ttr(df_role_1['Tweet'])
df_role_2_ttr = calculate_overall_ttr(df_role_2['Tweet'])
df_role_3_ttr = calculate_overall_ttr(df_role_3['Tweet'])
df_role_4_ttr = calculate_overall_ttr(df_role_4['Tweet'])
df_role_5_ttr = calculate_overall_ttr(df_role_5['Tweet'])
df_single_no_role_ttr = calculate_overall_ttr(df_single_no_role['Tweet'])
df_single_role_1_ttr = calculate_overall_ttr(df_single_role_1['Tweet'])
df_single_role_1_news_ttr = calculate_overall_ttr(df_single_role_1_news['Tweet'])

# Getting the average length of tweets
df_average_length = calculate_average_length_nltk(df['Tweet'])
df_no_role_average_length = calculate_average_length_nltk(df_no_role['Tweet'])
df_no_role_news_average_length = calculate_average_length_nltk(df_no_role_news['Tweet'])
df_role_1_average_length = calculate_average_length_nltk(df_role_1['Tweet'])
df_role_2_average_length = calculate_average_length_nltk(df_role_2['Tweet'])
df_role_3_average_length = calculate_average_length_nltk(df_role_3['Tweet'])
df_role_4_average_length = calculate_average_length_nltk(df_role_4['Tweet'])
df_role_5_average_length = calculate_average_length_nltk(df_role_5['Tweet'])
df_single_no_role_average_length = calculate_average_length_nltk(df_single_no_role['Tweet'])
df_single_role_1_average_length = calculate_average_length_nltk(df_single_role_1['Tweet'])
df_single_role_1_news_average_length = calculate_average_length_nltk(df_single_role_1_news['Tweet'])

# Print the DataFrame with TTR values
print("Overall TTR")
print(f"Original: {df_ttr}")
print(f"No role: {df_no_role_ttr}")
print(f"No role news: {df_no_role_news_ttr}")
print(f"Role 1: {df_role_1_ttr}")
print(f"Role 2: {df_role_2_ttr}")
print(f"Role 3: {df_role_3_ttr}")
print(f"Role 4: {df_role_4_ttr}")
print(f"Role 5: {df_role_5_ttr}")
print(f"Single label no role: {df_single_no_role_ttr}")
print(f"Single label role 1: {df_single_role_1_ttr}")
print(f"Single label role 1 news: {df_single_role_1_news_ttr}")

# Print the average lengths
print("\nAverage Lengths")
print(f"Original: {df_average_length}")
print(f"No role: {df_no_role_average_length}")
print(f"No role news: {df_no_role_news_average_length}")
print(f"Role 1: {df_role_1_average_length}")
print(f"Role 2: {df_role_2_average_length}")
print(f"Role 3: {df_role_3_average_length}")
print(f"Role 4: {df_role_4_average_length}")
print(f"Role 5: {df_role_5_average_length}")
print(f"Single label no role: {df_single_no_role_average_length}")
print(f"Single label role 1: {df_single_role_1_average_length}")
print(f"Single label role 1 news: {df_single_role_1_news_average_length}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Overall TTR
Original: 0.12422432544503532
No role: 0.05734749795702543
No role news: 0.06918589529223795
Role 1: 0.06678235002478929
Role 2: 0.0549834950977977
Role 3: 0.05440989099145864
Role 4: 0.06536685479733197
Role 5: 0.0766762913057427
Single label no role: 0.06015611244791361
Single label role 1: 0.06689404775002751
Single label role 1 news: 0.08661057252606548

Average Lengths
Original: 20.105586428780345
No role: 19.40578358208955
No role news: 24.488626023657872
Role 1: 18.72794800371402
Role 2: 19.094073377234242
Role 3: 19.566943674976915
Role 4: 18.147113594040967
Role 5: 22.940202391904325
Single label no role: 17.879328436516264
Single label role 1: 17.58027079303675
Single label role 1 news: 19.88


## Average number of labels per tweet

In [ ]:
def process_generated_data(generated_df):
    """
    Processes the DataFrame containing JSON strings in the first column.

    Parameters:
    generated_df (pd.DataFrame): DataFrame with JSON strings in the first column.

    Returns:
    pd.DataFrame: Processed DataFrame with boolean values for label columns.
    """
    # Convert the JSON strings in the first column to dictionaries
    parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

    # Create a new DataFrame from the parsed data
    generated_data = pd.DataFrame(parsed_data)

    # Drop rows with missing values
    generated_data = generated_data.dropna()
    generated_data = generated_data.drop_duplicates()

    # Define the list of label columns
    label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love',
                     'anger', 'disgust', 'pessimism', 'sadness', 'fear',
                     'surprise']

    # Convert label columns to boolean values
    generated_data[label_columns] = generated_data[label_columns].astype(bool)

    return generated_data

In [ ]:
def average_labels_per_tweet(df: pd.DataFrame) -> float:
    """
    Calculate the average number of labels per tweet in a multi-label classification dataframe.

    Parameters:
    df (pd.DataFrame): DataFrame where columns represent labels and values are 0 or 1.

    Returns:
    float: Average number of labels per tweet.
    """

    label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love',
                     'anger', 'disgust', 'pessimism', 'sadness', 'fear',
                     'surprise']

    # Count the number of labels per tweet
    labels_count_per_tweet = df[label_columns].sum(axis=1)

    # Calculate the average number of labels per tweet
    average_labels = labels_count_per_tweet.mean()

    return average_labels

df_no_role = pd.read_csv('generated_tweets_no_role.csv')
df_no_role = process_generated_data(df_no_role)

df_no_role_news = pd.read_csv('generated_tweets_no_role_news.csv')
df_no_role_news = process_generated_data(df_no_role_news)

df_role_1 = pd.read_csv('generated_tweets_role_1.csv')
df_role_1 = process_generated_data(df_role_1)

df_role_2 = pd.read_csv('generated_tweets_role_2.csv')
df_role_2 = process_generated_data(df_role_2)

df_role_3 = pd.read_csv('generated_tweets_role_3.csv')
df_role_3 = process_generated_data(df_role_3)

df_role_4 = pd.read_csv('generated_tweets_role_4.csv')
df_role_4 = process_generated_data(df_role_4)

df_role_5 = pd.read_csv('generated_tweets_role_5.csv')
df_role_5 = process_generated_data(df_role_5)

df_increased_multi = pd.read_csv('generated_tweets_increased_multi.csv')
df_increased_multi = process_generated_data(df_increased_multi)

print(f"No role: {average_labels_per_tweet(df_no_role)}")
print(f"No role news: {average_labels_per_tweet(df_no_role_news)}")
print(f"Role 1: {average_labels_per_tweet(df_role_1)}")
print(f"Role 2: {average_labels_per_tweet(df_role_2)}")
print(f"Role 3: {average_labels_per_tweet(df_role_3)}")
print(f"Role 4: {average_labels_per_tweet(df_role_4)}")
print(f"Role 5: {average_labels_per_tweet(df_role_5)}")
print(f"Increased multi: {average_labels_per_tweet(df_increased_multi)}")

No role: 4.229477611940299
No role news: 4.592356687898089
Role 1: 4.109563602599814
Role 2: 4.333960489181561
Role 3: 4.7276084949215145
Role 4: 3.211359404096834
Role 5: 4.632014719411224
Increased multi: 4.231643770724775
